In [1]:
import cv2
import os
import numpy as np
import mysql.connector
from datetime import datetime

# Initialize face recognizer
recognizer = cv2.face.LBPHFaceRecognizer_create()

dataset_path = r"C:\Users\Admin\Desktop\ML-project\CGC_Face_recognition\dataset"
face_images = []
labels = []
label_names = {}

# Load dataset
for i, directory in enumerate(os.listdir(dataset_path)):
    if os.path.isdir(os.path.join(dataset_path, directory)):  
        label_names[i] = directory
        for file in os.listdir(os.path.join(dataset_path, directory)):
            image_path = os.path.join(dataset_path, directory, file)
            face_img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            if face_img is not None:
                face_img = cv2.resize(face_img, (100, 100))
                face_images.append(face_img)
                labels.append(i)

# Train the recognizer
if len(face_images) > 0 and len(labels) > 0:
    recognizer.train(face_images, np.array(labels))
else:
    print("Error: Empty dataset")

# Connect to MySQL database
db_connection = mysql.connector.connect(
    host="localhost",
    user="root",  # Replace with your MySQL username
    password="...",  # Replace with your MySQL password
    database="face_recognition" # Replace with your MySQL database name
)
cursor = db_connection.cursor()

# Load Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Open webcam for real-time recognition
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        label, confidence = recognizer.predict(gray[y:y+h, x:x+w])

        if confidence < 100:  # Confidence threshold
            name = label_names[label]

            # Display name on the video frame
            cv2.putText(frame, name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

            # Log attendance
            current_time = datetime.now()
            query = """
                SELECT * FROM attendance 
                WHERE name = %s AND DATE(timestamp) = %s
            """
            cursor.execute(query, (name, current_time.date()))
            result = cursor.fetchall()

            if not result:
                insert_query = "INSERT INTO attendance (name, timestamp) VALUES (%s, %s)"
                cursor.execute(insert_query, (name, current_time))
                db_connection.commit()
                print(f"Attendance marked for {name} at {current_time}")
        else:
            cv2.putText(frame, "Unknown", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)

    cv2.imshow('Face Recognition', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()
cursor.close()
db_connection.close()


error: OpenCV(4.10.0) /Users/xperience/GHA-Actions-OpenCV/_work/opencv-python/opencv-python/opencv/modules/imgproc/src/color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
